In [1]:
import pandas as pd
import numpy as np
import datetime as dt # for date data type
import seaborn as sns # For creating plots
import geopandas as gdp
import matplotlib.pyplot as plt
import plotly.express as px
import holoviews as hv
import datetime
import numexpr
from platform import python_version

import panel as pn

pn.extension()
pn.extension('plotly')


In [2]:
df= pd.read_csv("df_clean.csv", parse_dates=['datetime']).set_index("datetime").sort_index()
df.head(5)

,Unnamed: 0,Juhtumi nr,Toimumisaeg,Isikuid,Hukkunuid,Vigastatuid,Sõidukeid,Aadress (PPA),Maja nr (PPA),Tänav (PPA),...,Ilmastik [1],Valgustus [1],Valgustus [2],GPS X,GPS Y,Asukoht,Nearest_Location,Temperature,lon,lat
datetime,,,,,,,,,,,,,,,,,,,,,
2011-01-05,2117,2490110002994,1/5/11,1,0,2,1,Jõgeva maakond VANA Põltsamaa vald,NaN,PIKKNURME - PÕLTSAMAA,...,Selged olud,Valge aeg,Valge aeg,6495324,621101,POINT (26.082246440862306 58.5809878216108),temp_tartu,-5.0,26.082246,58.580988
2011-01-08,2350,2502110056614,1/8/11,1,0,1,1,Pärnu maakond VANA Häädemeeste vald,NaN,IKLA - PIIRI,...,Selged olud,Valge aeg,Valge aeg,6415281,522787,POINT (24.38404570760267 57.87861953982567),temp_virtsu,-4.2,24.384046,57.878620
2011-01-08,455,2302110170106,1/8/11,2,0,1,2,Harju maakond Rae vald,NaN,TALLINN - TARTU - VÕRU - LUHAMAA,...,Selged olud,Valge aeg,Valge aeg,6582359,548106,POINT (24.846256904304 59.37649481762436),temp_tallinn,0.2,24.846257,59.376495
2011-01-11,644,2302110256697,1/11/11,2,1,0,3,Harju maakond Kiili vald,NaN,TALLINN - RAPLA - TÜRI,...,Selged olud,Valge aeg,Valge aeg,6579784,543958,POINT (24.772772270384877 59.35383139613875),temp_tallinn,2.2,24.772772,59.353831
2011-01-11,643,2302110256683,1/11/11,1,0,1,1,Harju maakond Tallinn Nõmme linnaosa Kadaka pst,NaN,Kadaka pst,...,Pilvised olud,Valge aeg,Valge aeg,6583734,536477,POINT (24.641931252760365 59.389994607712055),temp_tallinn,2.2,24.641931,59.389995


In [3]:
# Make suitable df for dashboard

df['year'] = df.index.year.astype('category')
df['month'] = df.index.month.astype('category')
df['day_of_week'] = df.index.day_name().astype('category')
df['hour_of_day'] = np.where(df.index.time != datetime.time(0, 0), df.index.hour.astype('category'), np.nan)
df['Liiklusõnnetuse liik [3]']=df['Liiklusõnnetuse liik [3]'].astype('category')
df['Lubatud sõidukiirus (PPA)']=df['Lubatud sõidukiirus (PPA)'].astype('category')


In [4]:
df.columns

Index(['Unnamed: 0', 'Juhtumi nr', 'Toimumisaeg', 'Isikuid', 'Hukkunuid',
       'Vigastatuid', 'Sõidukeid', 'Aadress (PPA)', 'Maja nr (PPA)',
       'Tänav (PPA)', 'Ristuv tänav (PPA)', 'Maakond (PPA)',
       'Omavalitsus (PPA)', 'Asustus (PPA)', 'Asula',
       'Liiklusõnnetuse liik [1]', 'Liiklusõnnetuse liik [3]',
       'Kergliikurijuhi osalusel', 'Jalakäija osalusel', 'Kaassõitja osalusel',
       'Maastikusõiduki juhi osalusel',
       'Eaka (65+) mootorsõidukijuhi osalusel', 'Bussijuhi osalusel',
       'Veoautojuhi osalusel', 'Ühissõidukijuhi osalusel',
       'Sõiduautojuhi osalusel', 'Mootorratturi osalusel',
       'Mopeedijuhi osalusel', 'Jalgratturi osalusel', 'Alaealise osalusel',
       'Turvavarustust mitte kasutanud isiku osalusel',
       'Esmase juhiloa omaniku osalusel', 'Mootorsõidukijuhi osalusel',
       'Tüüpskeemi nr', 'Tüüpskeem [2]', 'Tee tüüp [1]', 'Tee tüüp [2]',
       'Tee element [1]', 'Tee element [2]', 'Tee objekt [2]', 'Kurvilisus',
       'Tee tasa

## Siia kogun nupud

In [5]:
min = pd.to_datetime(df.index.min())
max =pd.to_datetime(df.index.max())

# Slider kaardile aja perioodi muutmiseks
slider = pn.widgets.DateRangeSlider(
    name='Ajavahemik',
    start=min, end=max,
    value=(min, max),
    step=10000
)

#datetime_range_slider

In [6]:
# valikunupud perioodi joonistele

perioodid = pn.widgets.RadioButtonGroup(
    name='Periood', options=['Aastad', 'Kuud', 'Päevad', 'Tunnid'], button_type='default')

perioodid

BokehModel(combine_events=True, render_bundle={'docs_json': {'e5ff0b11-11a8-463a-bbb7-7474f75dcaf5': {'defs': …

In [7]:
# valikunupud õnnetuste joonistele

õnnetused = pn.widgets.RadioButtonGroup(
    name='Õnnetused', options=['Osalejad', 'Õnnetuse liik'], button_type='default')

õnnetused

BokehModel(combine_events=True, render_bundle={'docs_json': {'2b4d5e3a-4ddf-4aac-9777-fb52cb988bcf': {'defs': …

In [8]:
# valikunupud liiklusolude joonistele

olukord = pn.widgets.RadioButtonGroup(
    name='Liiklusolukord', options=['Tee element', 'Tee objekt', 'Kurvilisus', 'Tee tasasus', 'Tee seisund', 'Teekate', 'Teekatte seisund', 'Piirkiirus'], button_type='default')

olukord

BokehModel(combine_events=True, render_bundle={'docs_json': {'eb37ce7d-d731-4566-8c8a-7ddcddbed2d1': {'defs': …

In [9]:
# multiselect widget for participant type
# pole kasutusel

multi_choice = pn.widgets.MultiChoice(name='Liiklusõnnetuse osalised', value=[],
    options=['Jalakäija osalusel', 'Kergliikurijuhi osalusel', 'Kaassõitja osalusel', 'Maastikusõiduki juhi osalusel', "Eaka (65+) mootorsõidukijuhi osalusel",  "Bussijuhi osalusel", "Veoautojuhi osalusel", "Ühissõidukijuhi osalusel", "Sõiduautojuhi osalusel", "Mootorratturi osalusel", "Mopeedijuhi osalusel", "Jalgratturi osalusel", "Alaealise osalusel", "Turvavarustust mitte kasutanud isiku osalusel", "Esmase juhiloa omaniku osalusel", "Mootorsõidukijuhi osalusel"])

pn.Column(multi_choice, height=400)
#multi_choice

BokehModel(combine_events=True, render_bundle={'docs_json': {'9d2509d3-a6bb-4992-bcac-f182cf6082ec': {'defs': …

## Siin teen joonised

In [10]:
# suured numbrid - hukkunud, vigastatud ja õnnetusi
@pn.depends( pn.widgets.DateRangeSlider.param.value_throttled)
def statistika(slider_value, veerg):
    if veerg=="Õnnetusi":
        return pn.indicators.Number(name=veerg, value=df.loc[pd.to_datetime(slider_value[0]):pd.to_datetime(slider_value[1]),'Juhtumi nr'].count(), font_size = "36pt", width=150)  
    else:
        return pn.indicators.Number(name=veerg, value=df.loc[pd.to_datetime(slider_value[0]):pd.to_datetime(slider_value[1]), veerg].sum(), font_size = "36pt", width=150)

pn.Row(statistika(slider.value, 'Õnnetusi'),statistika(slider.value, 'Õnnetusi'))

BokehModel(combine_events=True, render_bundle={'docs_json': {'8a95f0bf-1c75-4cb0-b0dc-bd1b5d041f04': {'defs': …

In [11]:
# liiklusõnnetused liiklusolude kaupa kaupa
@pn.depends( pn.widgets.RadioButtonGroup.param.value)   
def liiklusolud(liik, slider_value):
    df_kaart=df.loc[pd.to_datetime(slider_value[0]):pd.to_datetime(slider_value[1])]    
    

    # Counting the number of accidents for each time component
    element = df_kaart['Tee element [2]'].value_counts().sort_index().sort_values()
    objekt = df_kaart['Tee objekt [2]'].value_counts().sort_index().sort_values()
    kurvilisus = df_kaart['Kurvilisus'].value_counts().sort_index().sort_values()
    tasasus = df_kaart['Tee tasasus'].value_counts().sort_index().sort_values()
    seisund = df_kaart['Tee seisund'].value_counts().sort_index().sort_values()
    teekate = df_kaart['Teekate'].value_counts().sort_index().sort_values()
    teekatte_seisund = df_kaart['Teekatte seisund [2]'].value_counts().sort_index().sort_values()
    piirkiirus = df_kaart['Lubatud sõidukiirus (PPA)'].value_counts().sort_index().sort_values()


    
# Creating bar plots using Plotly
    if liik == "Tee element":
        fig = px.bar(y=element.index, x=element.values, labels={'x': 'Õnnetuste arv'}, title='Õnnetuste arv erinevatel tee elementidel')
    elif liik == "Tee objekt": 
        fig = px.bar(y=objekt.index, x=objekt.values, labels={'x': 'Õnnetuste arv'}, title='Õnnetuste arv erinevatel tee objektidel')
    elif liik == "Kurvilisus": 
        fig = px.bar(y=kurvilisus.index, x=kurvilisus.values, labels={'x': 'Õnnetuste arv'}, title='Õnnetuste arv erineva kurvilisusega teedel')
    elif liik == "Tee tasasus": 
        fig = px.bar(y=tasasus.index, x=tasasus.values, labels={'x': 'Õnnetuste arv'}, title='Õnnetuste arv erinevate  tasasutega teedel')
    elif liik == "Tee seisund": 
        fig = px.bar(y=seisund.index, x=seisund.values, labels={'x': 'Õnnetuste arv'}, title='Õnnetuste arv erineva seisundiga teedel')
    elif liik == "Teekate": 
        fig = px.bar(y=teekate.index, x=teekate.values, labels={'x': 'Õnnetuste arv'}, title='Õnnetuste arv erineva teekattega teedel')
    elif liik == "Teekatte seisund": 
        fig = px.bar(y=teekatte_seisund.index, x=teekatte_seisund.values, labels={'x': 'Õnnetuste arv'}, title='Õnnetuste arv erineva teekatte seisundiga teedel')
    elif liik == "Piirkiirus": 
        fig = px.bar(y=piirkiirus.index, x=piirkiirus.values, labels={'x': 'Õnnetuste arv'}, title='Õnnetuste arv erineva piirkiirusega teedel')
        
        
    return fig

liiklusolud(olukord.value, slider.value)

In [12]:
# liiklusõnnetused osaleja tüüpide kaupa
@pn.depends( pn.widgets.RadioButtonGroup.param.value)   
def osalejad(liik, slider_value):
    df_kaart=df.loc[pd.to_datetime(slider_value[0]):pd.to_datetime(slider_value[1])]    
    options=['Jalakäija osalusel', 'Kergliikurijuhi osalusel', 'Kaassõitja osalusel', 'Maastikusõiduki juhi osalusel', "Eaka (65+) mootorsõidukijuhi osalusel",  "Bussijuhi osalusel", "Veoautojuhi osalusel", "Ühissõidukijuhi osalusel", "Sõiduautojuhi osalusel", "Mootorratturi osalusel", "Mopeedijuhi osalusel", "Jalgratturi osalusel", "Alaealise osalusel", "Turvavarustust mitte kasutanud isiku osalusel", "Esmase juhiloa omaniku osalusel", "Mootorsõidukijuhi osalusel"]

    # Count the number of accidents for each participant type
    df_count = df_kaart[options].apply(pd.Series.value_counts).T.reset_index().sort_values(1.0)
    # Counting the number of accidents for each time component
    õnnetusi = df_kaart['Liiklusõnnetuse liik [3]'].value_counts().sort_index().sort_values()


    # Create the plot using Plotly Express
    fig1 = px.bar(df_count, y='index', x=1.0, title='Õnnetustes osalejate arv')
    fig1.update_layout(
    xaxis_title='Kokku',
    yaxis_title='')
    
# Creating bar plots using Plotly
    if liik == "Osalejad":
        return fig1
    elif liik == "Õnnetuse liik": 
        fig = px.bar(y=õnnetusi.index, x=õnnetusi.values, labels={'x': 'Õnnetuste arv'}, title='Erinevat liiki õnnetuste arv')
        return fig

osalejad(õnnetused.value, slider.value)

In [13]:
# õnnetuste liigid

@pn.depends( pn.widgets.DateRangeSlider.param.value_throttled)
def onnetused_joonis(slider_value):
    df_kaart=df.loc[pd.to_datetime(slider_value[0]):pd.to_datetime(slider_value[1])]
    
    # Counting the number of accidents for each time component
    õnnetusi = df_kaart['Liiklusõnnetuse liik [3]'].value_counts().sort_index().sort_values()


    # Creating bar plots using Plotly
    fig = px.bar(y=õnnetusi.index, x=õnnetusi.values, labels={'y': 'Õnnetuse liik','x': 'Õnnetuste arv'}, title='Erinevat liiki õnnetuste arv')
    
    return fig

onnetused_joonis(slider.value)

In [14]:
# Extracting year, month, day of week, and hour of day from the 'time_of_accident' column

#@pn.depends( pn.widgets.Button.param.button_type.objects)
@pn.depends( pn.widgets.RadioButtonGroup.param.value)
def perioodid_joonis(perioodid, slider_value):
    df_kaart=df.loc[pd.to_datetime(slider_value[0]):pd.to_datetime(slider_value[1])]
    
    # Counting the number of accidents for each time component
    Aastad = df_kaart['year'].value_counts().sort_index()
    Kuud = df_kaart['month'].value_counts().sort_index()
    Päevad = df_kaart['day_of_week'].value_counts().sort_index()
    Tunnid =df_kaart['hour_of_day'].value_counts().sort_index()

    # Creating bar plots using Plotly
    if perioodid == "Aastad":
        fig = px.bar(x=Aastad.index, y=Aastad.values, labels={'x': 'Aasta', 'y': 'Õnnetuste arv'}, title='Õnnetusi aasta kaupa')
    elif perioodid == "Kuud":
        fig = px.bar(x=Kuud.index, y=Kuud.values, labels={'x': 'Kuu','y': 'Õnnetuste arv'}, title='Õnnetusi kuude kaupa')
    elif perioodid == "Päevad":
        fig = px.bar(x=Päevad.index, y=Päevad.values, labels={'x': 'Nädalapäev','y': 'Õnnetuste arv'}, title='Õnnetusi päevade kaupa')
    elif perioodid == "Tunnid":
        fig = px.bar(x=Tunnid.index, y=Tunnid.values, labels={'x': 'Tund','y': 'Õnnetuste arv'}, title='Õnnetusi tundide kaupa')
    
    return fig

perioodid_joonis(perioodid.value, slider.value)

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 13071 entries, 2011-01-05 00:00:00 to 2021-12-31 23:45:00
Data columns (total 63 columns):
 #   Column                                         Non-Null Count  Dtype   
---  ------                                         --------------  -----   
 0   Unnamed: 0                                     13071 non-null  int64   
 1   Juhtumi nr                                     13071 non-null  object  
 2   Toimumisaeg                                    13071 non-null  object  
 3   Isikuid                                        13071 non-null  int64   
 4   Hukkunuid                                      13071 non-null  int64   
 5   Vigastatuid                                    13071 non-null  int64   
 6   Sõidukeid                                      13071 non-null  int64   
 7   Aadress (PPA)                                  13068 non-null  object  
 8   Maja nr (PPA)                                  6013 non-null   object  
 9   Täna

In [16]:
# Kaart liiklusõnnetustega

@pn.depends( pn.widgets.DateRangeSlider.param.value_throttled)
def kaart(slider_value):
    df_kaart=df.loc[pd.to_datetime(slider_value[0]):pd.to_datetime(slider_value[1])]
    fig = px.scatter_mapbox(df_kaart, lat ="lat", lon="lon", height=600, zoom=6.5)
    fig.update_layout(mapbox_style="open-street-map")
    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
    #fig.show()
    
    return fig
#kaart(lider.value)

In [17]:
# Function to display current year
@pn.depends( pn.widgets.DateRangeSlider.param.value_throttled)
def tiitel(slider):
    return '### Liiklusõnnetuste statistika ajavahemikul '

## Dashboard

In [ ]:
dashboard = pn.WidgetBox(pn.Row(
                             pn.Column(
                                 pn.Row(tiitel, slider),
                                 pn.Row(pn.bind(kaart, slider)),
                                 pn.Row(perioodid),
                                 pn.bind(perioodid_joonis, perioodid, slider), align="start",          
                                 sizing_mode="stretch_width"), 
                             pn.Column(
                                 pn.Row(
                                     pn.bind(statistika, slider, 'Õnnetusi'),
                                     pn.bind(statistika, slider, 'Vigastatuid'),
                                     pn.bind(statistika, slider, 'Hukkunuid')
                                 ),
                                 õnnetused,
                                 pn.bind(osalejad, õnnetused, slider),
                                 olukord,
                                 pn.bind(liiklusolud, olukord, slider)
                             ))
                        )
dashboard.show()

Launching server at http://localhost:61161
